<a href="https://colab.research.google.com/github/saadallahchaima/UnityProject_ActiveLearning/blob/main/stepDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
!pip install onnxruntime


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 60.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.7 MB/s eta 0:00:00


In [7]:
!pip install datasets


In [8]:
from datasets import load_dataset

ds = load_dataset("codymlewis/HAR")

In [17]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
import onnx
import onnxruntime as ort
import torch
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType


In [9]:
# Display the first few rows of the dataset
print(ds['train'].features)
print(ds['train'][:5])  # Display first 5 records


{'features': Sequence(feature=Value(dtype='float32', id=None), length=561, id=None), 'labels': ClassLabel(names=['WALKING', 'WALKING_UPSTAIRS', 'WALKING_DOWNSTAIRS', 'SITTING', 'STANDING', 'LAYING', 'STAND_TO_SIT', 'SIT_TO_STAND', 'SIT_TO_LIE', 'LIE_TO_SIT', 'STAND_TO_LIE', 'LIE_TO_STAND'], id=None), 'subject id': Value(dtype='uint8', id=None)}
{'features': [[0.04357967525720596, -0.0059702214784920216, -0.035054344683885574, -0.9953811764717102, -0.988365888595581, -0.9373819828033447, -0.9950070381164551, -0.9888156056404114, -0.9533252120018005, -0.794796347618103, -0.7448928356170654, -0.6484472155570984, 0.8417955636978149, 0.7084401845932007, 0.6517164707183838, -0.975752055644989, -0.9999502301216125, -0.9998883605003357, -0.9980136752128601, -0.9939993023872375, -0.9919795989990234, -0.9709700345993042, -0.5470953583717346, -0.700973629951477, -0.622697114944458, 0.9218841791152954, -0.7194828987121582, 0.34216800332069397, -0.1613176465034485, 0.26604896783828735, -0.274351447

In [20]:
df = pd.DataFrame(ds['train'])

In [21]:
# Display the first few rows and column names
print(df.head())
print(df.columns)

                                            features  labels  subject id
0  [0.04357967525720596, -0.0059702214784920216, ...       4           1
1  [0.03948003798723221, -0.0021312758326530457, ...       4           1
2  [0.0399777814745903, -0.005152716301381588, -0...       4           1
3  [0.03978455811738968, -0.011808778159320354, -...       4           1
4  [0.038758136332035065, -0.0022885329090058804,...       4           1
Index(['features', 'labels', 'subject id'], dtype='object')


In [22]:
# Check the shape of the features in the first few rows
df['features'].apply(len).value_counts()



,count
features,
561,7767


In [23]:
# Preview the dataset
print(df.head())

                                            features  labels  subject id
0  [0.04357967525720596, -0.0059702214784920216, ...       4           1
1  [0.03948003798723221, -0.0021312758326530457, ...       4           1
2  [0.0399777814745903, -0.005152716301381588, -0...       4           1
3  [0.03978455811738968, -0.011808778159320354, -...       4           1
4  [0.038758136332035065, -0.0022885329090058804,...       4           1


In [24]:
# Flatten the features column (list of floats) and create a new DataFrame
features = pd.DataFrame(df['features'].tolist())  # Each list in 'features' becomes a row


In [25]:
# Combine features with labels and subject id
acc_data_cleaned = pd.concat([features, df['labels'], df['subject id']], axis=1)


In [26]:
# Rename the columns for easier access (optional)
acc_data_cleaned.columns = [f'acc_{i}' for i in range(features.shape[1])] + ['labels', 'subject id']


In [27]:
# Verify the cleaned data
print(acc_data_cleaned.head())

      acc_0     acc_1     acc_2     acc_3     acc_4     acc_5     acc_6  \
0  0.043580 -0.005970 -0.035054 -0.995381 -0.988366 -0.937382 -0.995007   
1  0.039480 -0.002131 -0.029067 -0.998348 -0.982945 -0.971273 -0.998702   
2  0.039978 -0.005153 -0.022651 -0.995482 -0.977314 -0.984760 -0.996415   
3  0.039785 -0.011809 -0.028916 -0.996194 -0.988569 -0.993256 -0.996994   
4  0.038758 -0.002289 -0.023863 -0.998241 -0.986774 -0.993115 -0.998216   

      acc_7     acc_8     acc_9  ...   acc_553   acc_554   acc_555   acc_556  \
0 -0.988816 -0.953325 -0.794796  ... -0.713308 -0.112754  0.030400 -0.464761   
1 -0.983315 -0.974000 -0.802537  ... -0.860677  0.053477 -0.007435 -0.732626   
2 -0.975835 -0.985973 -0.798477  ... -0.761847 -0.118559  0.177899  0.100699   
3 -0.988526 -0.993135 -0.798477  ... -0.491604 -0.036788 -0.012892  0.640011   
4 -0.986479 -0.993825 -0.801982  ... -0.702490  0.123320  0.122542  0.693578   

    acc_557   acc_558   acc_559   acc_560  labels  subject id  
0 -0

In [28]:
# Separate features and labels
X = acc_data_cleaned.drop(columns=['labels', 'subject id'])  # All feature columns
y = acc_data_cleaned['labels']  # Your activity labels

# Check the shapes
print("Shape of X (features):", X.shape)  # Should show the number of samples and features
print("Shape of y (labels):", y.shape)     # Should match the number of samples


Shape of X (features): (7767, 561)
Shape of y (labels): (7767,)


In [29]:
from sklearn.model_selection import train_test_split

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Verify the split
print("Training set size:", X_train.shape[0])  # Should be 80% of the total
print("Testing set size:", X_test.shape[0])    # Should be 20% of the total


Training set size: 6213
Testing set size: 1554


In [30]:
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)


RandomForestClassifier(random_state=42)

In [31]:
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.98      0.98      0.98       245
           1       0.98      0.99      0.98       215
           2       0.97      0.98      0.98       197
           3       0.97      0.94      0.95       259
           4       0.95      0.97      0.96       285
           5       1.00      1.00      1.00       283
           6       0.80      0.89      0.84         9
           7       1.00      0.80      0.89         5
           8       0.75      0.60      0.67        15
           9       0.71      0.83      0.77        12
          10       0.68      0.72      0.70        18
          11       0.86      0.55      0.67        11

    accuracy                           0.97      1554
   macro avg       0.89      0.85      0.87      1554
weighted avg       0.97      0.97      0.96      1554

[[240   2   3   0   0   0   0   0   0   0   0   0]
 [  1 213   1   0   0   0   0   0   0   0   0   0]
 [  2   2 193   0   0   0   0  

In [32]:
!pip install skl2onnx


In [33]:
# Convert the RandomForest model to ONNX format
initial_type = [('float_input', FloatTensorType([None, X_train.shape[1]]))]
onnx_model = convert_sklearn(model, initial_types=initial_type)

# Save the ONNX model to file
onnx_model_path = "step_detection_model.onnx"
with open(onnx_model_path, "wb") as f:
    f.write(onnx_model.SerializeToString())

print(f"Model exported to {onnx_model_path}")


Model exported to step_detection_model.onnx


In [34]:
print(acc_data_cleaned['labels'].unique())


[ 4  6  3  7 10  5  9  8 11  0  2  1]


In [35]:
# Hypothetical mapping of labels
label_mapping = {
    0: "Standing",
    1: "Sitting",
    2: "Lying",
    3: "Walking",
    4: "Running",
    5: "Jumping",
    6: "Stairs Up",
    7: "Stairs Down",
    8: "Cycling",
    9: "Swimming",
    10: "Cooking",
    11: "Other"
}

# Display unique labels and their corresponding activities
unique_labels = acc_data_cleaned['labels'].unique()
activity_names = [label_mapping[label] for label in unique_labels]

# Print unique labels with activity names
for label, activity in zip(unique_labels, activity_names):
    print(f"Label: {label}, Activity: {activity}")


Label: 4, Activity: Running
Label: 6, Activity: Stairs Up
Label: 3, Activity: Walking
Label: 7, Activity: Stairs Down
Label: 10, Activity: Cooking
Label: 5, Activity: Jumping
Label: 9, Activity: Swimming
Label: 8, Activity: Cycling
Label: 11, Activity: Other
Label: 0, Activity: Standing
Label: 2, Activity: Lying
Label: 1, Activity: Sitting


In [36]:
y_pred = model.predict(X_test)  # X_test contains your test feature set


In [38]:
import numpy as np
# Count occurrences of each predicted class
unique_pred, counts_pred = np.unique(y_pred, return_counts=True)

# Map predicted labels to activity names (if necessary)
predicted_activity_names = [label_mapping[label] for label in unique_pred]

# Display counts for Walking and Running
for label, count in zip(unique_pred, counts_pred):
    activity_name = label_mapping[label]
    print(f"Predicted Activity: {activity_name}, Count: {count}")

# Specifically check for steps
steps_count = counts_pred[np.isin(unique_pred, [3, 4])].sum()  # Assuming 3 = Walking, 4 = Running
print(f"Total Steps Predicted (Walking + Running): {steps_count}")


Predicted Activity: Standing, Count: 245
Predicted Activity: Sitting, Count: 218
Predicted Activity: Lying, Count: 198
Predicted Activity: Walking, Count: 252
Predicted Activity: Running, Count: 291
Predicted Activity: Jumping, Count: 284
Predicted Activity: Stairs Up, Count: 10
Predicted Activity: Stairs Down, Count: 4
Predicted Activity: Cycling, Count: 12
Predicted Activity: Swimming, Count: 14
Predicted Activity: Cooking, Count: 19
Predicted Activity: Other, Count: 7
Total Steps Predicted (Walking + Running): 543


In [40]:
import torch
import torch.nn as nn
import torch.onnx

# Define a simple neural network
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(561, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 12)  # Assuming 12 activity labels

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return torch.log_softmax(self.fc3(x), dim=1)

# Instantiate and train the model, or load pre-trained weights if available
model = SimpleNN()

# Export the model to ONNX
dummy_input = torch.randn(1, 561)  # Adjust input dimensions as per the dataset
onnx_model_path = "simple_nn_model.onnx"
torch.onnx.export(model, dummy_input, onnx_model_path)
